In [1]:
import pandas as pd
import numpy as np

In [2]:
d1 = {'Social_media_followers':[1000000, np.nan, 2000000, 1310000, 1700000, np.nan, 4100000, 1600000, 2200000, 1000000],
'Sold_out':[1,0,0,1,0,0,0,1,0,1]}

In [9]:
df1=pd.DataFrame(d1)

In [12]:
X1=df1[['Social_media_followers']]
y1=df1[['Sold_out']]

In [15]:
from sklearn.model_selection import train_test_split
X1_train,X1_test,y1_train,y1_test=train_test_split(X1,y1,test_size=.3,random_state=19)

In [16]:
X1_train.shape

(7, 1)

In [17]:
X1_test.shape

(3, 1)

In [21]:
from sklearn.impute import SimpleImputer 
imputor=SimpleImputer(strategy='mean')

In [24]:
from sklearn.linear_model import LogisticRegression

In [25]:
lr=LogisticRegression()

In [26]:
from sklearn.pipeline import make_pipeline

In [27]:
pipe1=make_pipeline(imputor,lr)

In [28]:
pipe1.fit(X1_train,y1_train)

/opt/anaconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('logisticregression', LogisticRegression())])

In [29]:
pipe1.score(X1_train,y1_train)

1.0

In [30]:
pipe1.score(X1_test,y1_test)

0.6666666666666666

In [32]:
pipe1.named_steps.simpleimputer.statistics_

array([2051666.66666667])

In [34]:
pipe1.named_steps.logisticregression.coef_

array([[-9.72872137e-05]])

In [35]:
d2 = {'Genre':['Rock', 'Metal', 'Bluegrass', 'Rock', np.nan, 'Rock', 'Rock', np.nan, 'Bluegrass', 'Rock'],
'Social_media_followers':[1000000, np.nan, 2000000, 1310000, 1700000, np.nan, 4100000, 1600000, 2200000, 1000000],
'Sold_out':[1,0,0,1,0,0,0,1,0,1]}

In [36]:
df=pd.DataFrame(data=d2)

In [37]:
df

,Genre,Social_media_followers,Sold_out
0,Rock,1000000.0,1
1,Metal,NaN,0
2,Bluegrass,2000000.0,0
3,Rock,1310000.0,1
4,NaN,1700000.0,0
5,Rock,NaN,0
6,Rock,4100000.0,0
7,NaN,1600000.0,1
8,Bluegrass,2200000.0,0
9,Rock,1000000.0,1


In [38]:
X=df.iloc[:, 0:2]
y=df.iloc[:,2]

In [39]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.3,random_state=19)

In [40]:
num_cols=['Social_media_followers']
cat_cols=['Genre']

In [45]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [51]:
num_pipeline=Pipeline(steps=[
    ('impute',SimpleImputer(strategy='mean')),
    ('scale',StandardScaler())
    ])

In [52]:
from sklearn.preprocessing import OneHotEncoder

In [54]:
cat_pipeline=Pipeline(steps=[
    ('impute',SimpleImputer(strategy='most_frequent')),
    ('one-hot-encoder',OneHotEncoder(handle_unknown='ignore',sparse_output=False))
]
)

In [55]:
from sklearn.compose import ColumnTransformer

In [59]:
col_transformer=ColumnTransformer(transformers=[
    ('num_pipeline',num_pipeline,num_cols),
    ('cat_pipeline',cat_pipeline,cat_cols)
],
    remainder='drop',
    n_jobs=-1
)
   

In [61]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier()

In [62]:
pipefinal=make_pipeline(col_transformer,dtc)
pipefinal.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  ['Social_media_followers']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot-encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Genre'])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [63]:
pipefinal.score(X_test,y_test)

0.6666666666666666

In [64]:
import joblib

Exception ignored in: <function ResourceTracker.__del__ at 0x102fd9bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x10345dbc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x1031b1bc0>
Traceback (most recent call last

In [65]:
joblib.dump(pipefinal,"pipe.joblib")

['pipe.joblib']

In [67]:
pipefinal2=joblib.load("pipe.joblib")